## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import json
import pprint
import requests
from bs4 import BeautifulSoup

In [2]:
webPage = requests.get("http://www.alrb.org/datacleaning/covidcaseoutliers.html")

In [3]:
bs = BeautifulSoup(webPage.text, 'html.parser')

In [4]:
theadRows = bs.find('table', {'id':'tblDeaths'}).thead.find_all('th') # retrieving all elements nested within the 'th' elements for that table

In [5]:
labelcols = [j.get_text() for j in theadRows]
labelcols[0] = "rowheadings"

In [6]:
labelcols

['rowheadings',
 'Cases',
 'Deaths',
 'Cases per Million',
 'Deaths per Million',
 'population',
 'population_density',
 'median_age',
 'gdp_per_capita',
 'hospital_beds_per_100k']

In [10]:
rows = bs.find('table', {'id':'tblDeaths'}).tbody.find_all('tr')
datarows = []
labelrows = []
for row in rows:
    rowLabels = row.find('th').get_text()
    cells = row.find_all('td', {'class':'data'})
    if (len(rowLabels) > 3):
        labelrows.append(rowLabels)
    if (len(cells) > 0):
        cellvalues = [j.get_text() for j in cells]
        datarows.append(cellvalues)



[['9,394', '653', '214', '15', '43,851,043', '17', '29', '13,914', '1.9'],
 ['16,642', '668', '1848', '74', '9,006,400', '107', '44', '45,437', '7.4']]
['Algeria', 'Austria']


In [11]:
pprint.pprint(datarows[0:2])
pprint.pprint(labelrows[0:2])
for i in range(len(datarows)):
    datarows[i].insert(0, labelrows[i])

pprint.pprint(datarows[0:1])

[['9,394', '653', '214', '15', '43,851,043', '17', '29', '13,914', '1.9'],
 ['16,642', '668', '1848', '74', '9,006,400', '107', '44', '45,437', '7.4']]
['Algeria', 'Austria']
[['Algeria',
  '9,394',
  '653',
  '214',
  '15',
  '43,851,043',
  '17',
  '29',
  '13,914',
  '1.9']]


## Load Data into Pandas

In [12]:
totalDeaths = pd.DataFrame(datarows, columns=labelcols)
totalDeaths.head()

,rowheadings,Cases,Deaths,Cases per Million,Deaths per Million,population,population_density,median_age,gdp_per_capita,hospital_beds_per_100k
0,Algeria,"9,394",653,214,15,"43,851,043",17,29,"13,914",1.9
1,Austria,"16,642",668,1848,74,"9,006,400",107,44,"45,437",7.4
2,Bangladesh,"47,153",650,286,4,"164,689,383",1265,28,"3,524",0.8
3,Belgium,"58,381",9467,5037,817,"11,589,616",376,42,"42,659",5.6
4,Brazil,"514,849",29314,2422,138,"212,559,409",25,34,"14,103",2.2


In [13]:
totalDeaths.dtypes

rowheadings               object
Cases                     object
Deaths                    object
Cases per Million         object
Deaths per Million        object
population                object
population_density        object
median_age                object
gdp_per_capita            object
hospital_beds_per_100k    object
dtype: object

In [14]:
totalDeaths.columns = totalDeaths.columns.str.replace(" ", "_").str.lower()

In [15]:
for col in totalDeaths.columns[1:-1]:
    totalDeaths[col] = totalDeaths[col].str.replace("[^0-9]", "").astype('int64')

/var/folders/vh/9jy5nq5n6wl5cjp9y3m5l07w0000gn/T/ipykernel_19864/1469178818.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  totalDeaths[col] = totalDeaths[col].str.replace("[^0-9]", "").astype('int64')


In [16]:
totalDeaths['hospital_beds_per_100k'] = totalDeaths['hospital_beds_per_100k'].astype('float')
totalDeaths.head()

,rowheadings,cases,deaths,cases_per_million,deaths_per_million,population,population_density,median_age,gdp_per_capita,hospital_beds_per_100k
0,Algeria,9394,653,214,15,43851043,17,29,13914,1.9
1,Austria,16642,668,1848,74,9006400,107,44,45437,7.4
2,Bangladesh,47153,650,286,4,164689383,1265,28,3524,0.8
3,Belgium,58381,9467,5037,817,11589616,376,42,42659,5.6
4,Brazil,514849,29314,2422,138,212559409,25,34,14103,2.2
